In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Import the necessary libraries
import matplotlib.pyplot as plt
from matplotlib import pyplot
import numpy as np
import re
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Embedding, SpatialDropout1D, BatchNormalization

# from tensorflow.keras.layers.normalization import BatchNormalization


import wordcloud



In [3]:
# Make results reproducible - set random seed
from numpy.random import seed
seed(42)

In [4]:
negative_file = "/kaggle/input/movie-book-review/negative.txt"
positive_file = "/kaggle/input/movie-book-review/positive.txt"

In [5]:
# Do not modify - helper function to load and preprocess data
def filter_words(line):    
    line = re.sub(r'[^\w\s]','',line.rstrip())
    words = line.split(" ") 
    words = [i.lower() for i in words if i]      
    return " ".join(words)

def load_data(filename):
    thefile = open(filename, 'r') 
    lines = thefile.readlines() 

    data = []
    for l in range(0,len(lines)): 
        if(lines[l-1].strip() == "<title>"): 
            theline = filter_words(lines[l])
            if(len(theline) < 50):
                data.append(theline)            
            
    return data

# Helper function to convert categorical data to class label
def to_word_label(y):
    y = to_class(y)   
    return ["positive" if i==0 else "negative" for i in y]

# Helper function to convert class label to numeric label
def to_numeric_label(y):
  return [0 if i=="positive" else 1 for i in word_labels]

# Helper function: this function needs to be called before sending arrays to sklearn metrics,
# it converts back to class form from categorical form. ie: [1,0] --> 0, [0,1] --> 1
def to_class(y):
    return np.argmax(y,axis=1)

In [6]:
positive = load_data(positive_file)
negative = load_data(negative_file)

#to view the first 10 positive review text and first 10 negative reviews
print(positive[0:10])
print("\n")
print(negative[0:10])

In [7]:
# Do not modify - Combines the positive and negative reviews into a single list and create labels
data = positive + negative
word_labels = ["positive"] * len(positive) + ["negative"] * len(negative) 

# Converts labels to numbers in one-hot encoding - [1, 0] (positive) or [0, 1] (negative)
# from keras.utils import to_categorical
from tensorflow.keras.utils import to_categorical
labels  = to_categorical(to_numeric_label(word_labels))
print(type(data))

In [8]:
# Write some code to investigate the dataset. 
# - Calculate and report the mean review size, its standard deviation and create a boxplot.

#already loaded the matplotlib libraries at the top

count_words = []
count = 0

for i in data:
    word_list = i.split()
    number_of_words = len(word_list)
    count_words.append(number_of_words)
#     count_words += number_of_words
    count += 1

#Adding the words together
add = sum(count_words)
avg = add/count
print('The mean review word size is : {:.1f}'.format(avg))

std = np.std(count_words)
print("The standard deviation is calculated to: {:.1f}".format(std))

plt.boxplot(count_words)
plt.grid()
plt.show()

# - Calculate the number of unique words in the dataset

# A set is a collection in which all elements are unique.
unique_words = set(data)

unique_word_count = len(unique_words)
# display the number of unique words 
print("The number of unique words was calculated to be: {}".format(unique_word_count))

# - Perform any other dataset investigation that you feel would be valuable

#plot a wordcloud of positive and negative book reviews  

from wordcloud import WordCloud 

wordcloud = WordCloud(width = 1000, height = 500).generate(" ".join(data))
print("WORD CLOUD DIAGRAM")
plt.figure(figsize=(15,8))
plt.imshow(wordcloud)
plt.axis("off")
plt.savefig("your_file_name"+".png", bbox_inches='tight')
plt.show()
plt.close()



### The word cloud above displays some of the most frequent words found in the dataset.

In [9]:
# Do not modify - Tokenize the vocabulary 
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=25)

tokenizer.fit_on_texts(data) #create the vocabularry

tokenized_data = tokenizer.texts_to_sequences(data) #tokenize the data using the vocabulary

vocab_size = len(tokenizer.word_index) + 1 

# Compare a sample of the data before and after tokenization
print(data[0:5])
print(tokenized_data[0:5])
#display the length of the tokenized data
print(len(tokenized_data))

# Pre-processing

In [10]:
# Pre-processing
# Write some code to pre-process the data so that each review is the same length
# Put the padding at the end of the sequences

padded = pad_sequences(tokenized_data, maxlen = 4, padding = "post")

#display the first five padded sequence
print("Padded tokenized_data:\n {}".format(padded[ :5]))

In [11]:
# Write some code to split the data into a training and test set. Make sure you shuffle the data. Use 20% for the test set.
y = labels
X = padded

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True, random_state = 42)


In [12]:

from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score

# Fill in the following function so it
# - makes a prediction for the test set given the model
# - reports the precision, recall and f1 score. Also print the confusion matrix. 
# You will need to use the helper to_class function to convert y_pred and y_test before supplying them to the sklearn functions.

def assess_model(model, X_test, y_test):
    
    # use the helper to_class function to convert
    # y_pred and y_test before supplying them to the sklearn functions.
    y_test = to_class(y_test)
    
    #Get the predictions for y_pred
    #makes a prediction for the test set given the model
    y_pred = model.predict(X_test)
       
    # Calculate the confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)

    #Evaluate precision_score
    print()
    print(precision_score(y_test, y_pred))
    #Display the recall_score
    print()
    print(recall_score(y_test, y_pred))
    #Display the f1_score
    print(f1_score(y_test, y_pred))

# Build and tune model

Define network architecture

In [13]:
# initializing an object from TensorFlow’s Sequential class
# Build the sequencial model of recurrent neural network (RNN)
model = Sequential()

# output_dim can be any value
# output_dim is defined as the number of dimensions we wish to embed into
#creating and adding the embedding layer
embedding_layer = Embedding(input_dim = 1794, output_dim = 32, input_length = 4)
model.add(embedding_layer)


# adding SpatialDropout1D(0.2) layer
model.add(SpatialDropout1D(0.2))

# BatchNormalization()
BatchNormalization()

# add an lstm layer of 32
model.add(LSTM(32))

# adding a dense layer
model.add(Dense(2, activation = 'softmax'))



## Train model
#### TRAIN AND TUNE THE MODEL

In [14]:
# Compiling our neural network
# The compilation step of building a neural network is where we specify the neural net’s optimizer and loss function.
# model.compile(optimizer = 'adam', loss = 'binary_crossentropy')

# Fitting The Recurrent Neural Network On The Training Set
# Will specify epochs = 100 in this case.
# the size of batches (32) that the network will be trained in through each epoch.
# model.fit(X_train, y_train, output_dim = 10, epochs = 5, batch_size = 10)

# Defining the model

# function to determine the best output_dim
# creating an empty list
empty_list = []

lst = [10, 25, 50, 100]
for i in lst:
    
    # Initialize the classifier
    model = Sequential()
#     adding layers to the model
    embedding_layer = Embedding(input_dim = 1794, output_dim = i, input_length = 4)
    model.add(embedding_layer)
    model.add(SpatialDropout1D(0.3))
    BatchNormalization()
    model.add(LSTM(32))
    model.add(Dense(2, activation = 'softmax'))
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy')
    history = model.fit(X_train, y_train, epochs = 5, batch_size = 10, validation_steps = 30)
    model.summary()
    
    empty_list.append(model)
    

loss_value_model = empty_list[0] 

for model_loss in empty_list:
    
    if loss_value_model.loss > model_loss.loss:
        
        loss_value_model = model_loss
        

print("\n\n\n")        
loss_value_model.summary()    
    
    

    

Examine performance of model

Plot graphs for accuracy and loss

In [15]:
# Plot the loss_per_epoch
loss_per_epoch = model.history.history['loss']
plt.plot(range(len(loss_per_epoch)), loss_per_epoch)

In [16]:
# calling the assess_model function


In [17]:
#display the first 10 model predictions
predictins = model.predict(X_test)
predictins[0:10]

# Make a prediction

In [18]:
# This is a very small set of completed new data to use to make predictions.
prediction_data = ["this book is fabulous","i hated this book", "the best", "no good", "okay"]
tokenized = tokenizer.texts_to_sequences(prediction_data)
padded = pad_sequences(tokenized, padding = 'post', maxlen = 4)

# Supply this data to each of your models and see how it does. 
# You can call the helper function "to_word_label" to map the output of the model to the name of the
# class it was predicted to belong to.

#call filter_words function to load and preprocess data
#maxlen was changed to 4 

#display the predicted results of the model
print(to_word_label(padded))


# References

[1] https://www.freecodecamp.org/news/the-ultimate-guide-to-recurrent-neural-networks-in-python7/

[2] https://www.simplilearn.com/tutorials/deep-learning-tutorial/rnn

[3] https://www.freecodecamp.org/news/the-ultimate-guide-to-recurrent-neural-networks-in-python/

[4] https://python.tutorialink.com/generating-word-cloud-for-items-in-a-list-in-python/

[5] https://www.youtube.com/watch?v=jC0jXsX-33Q

[6] https://vitalflux.com/accuracy-precision-recall-f1-score-python-example/

[7] https://www.youtube.com/watch?v=S8tpSG6Q2H0

[8] https://towardsdatascience.com/step-by-step-guide-building-a-prediction-model-in-python-ac441e8b9e8b

[9] https://www.nbshare.io/notebook/249468051/How-To-Code-RNN-and-LSTM-Neural-Networks-in-Python/